In [ ]:
import tensorflow as tf

In [ ]:
def HHFB(inputs, scale=0.2):
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(inputs)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    c1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    c2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    c3 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    c4 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    c5 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    c6 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    c7 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    c8 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)    
    
    a1 = tf.keras.layers.Add()([c1, c2])
    a2 = tf.keras.layers.Add()([a1, c3])
    a3 = tf.keras.layers.Add()([a2, c4])
    a4 = tf.keras.layers.Add()([a3, c5])
    a5 = tf.keras.layers.Add()([a4, c6])
    a6 = tf.keras.layers.Add()([a5, c7])
    a7 = tf.keras.layers.Add()([a6, c8])
    
    x = tf.keras.layers.Concatenate(axis=-1)([c1, a1, a2, a3, a4, a5, a6, a7])
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1, padding='same', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Lambda(lambda t: t * scale)(x)
    x = tf.keras.layers.Add()([inputs, x])
    return x

In [ ]:
def CRM(inputs, filters, factor=2):
    x = tf.keras.layers.Conv2D(filters=filters*(factor**2), kernel_size=3, padding='same', kernel_initializer='he_normal')(inputs)
    x = tf.nn.depth_to_space(x, block_size=2)
    x = tf.keras.layers.Conv2D(filters=filters*(factor**2), kernel_size=3, padding='same', kernel_initializer='he_normal')(x)
    x = tf.nn.depth_to_space(x, block_size=2)
    x = tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    x = tf.keras.layers.Conv2D(filters=3, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    
    return x

In [ ]:
def RRFB(inputs, scale=0.2):
    x = HHFB(inputs)
    x = HHFB(x)    
    x = HHFB(x)
    x = tf.keras.layers.Lambda(lambda t: t * scale)(x)
    x = tf.keras.layers.Add()([inputs, x])
    return x

In [ ]:
def RFN():
    inputs = tf.keras.layers.Input(shape=(None, None, 3))
    x = tf.keras.layers.Rescaling(scale=1.0 / 255)(inputs)
    
    x = y = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(x)
    
    for _ in range(24):
        y = RRFB(inputs=y, scale=0.2)
        
    y = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_initializer='he_normal')(y)
    x = tf.keras.layers.Add()([x, y])
    
    x = CRM(inputs=x, filters=64)
    
    outputs = tf.keras.layers.Rescaling(scale=255)(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)